In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm import MLE, Laplace

In [2]:
df_en = pd.read_csv("D:/developer/github.com/projects/nlp_study/data/stackoverflow_ingles.csv")
df_pt = pd.read_csv("D:/developer/github.com/projects/nlp_study/data/stackoverflow_portugues.csv")
df_es = pd.read_csv("D:/developer/github.com/projects/nlp_study/data/stackoverflow_espanhol.csv")


In [3]:
display(df_en.head(5))
display(df_pt.head(5))
display(df_es.head(5))


,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940


,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


,Id,Título,Questão,Tags,Pontuação,Visualizações
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129


In [4]:
re.findall(r"<.*?>", df_pt["Questão"][0])

['<p>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_de_embaralhamento_criptogr%C3%A1fico" rel="noreferrer">',
 '</a>',
 '</em>',
 '</p>',
 '<p>',
 '<a href="http://pt.wikipedia.org/wiki/Ataque_de_for%C3%A7a_bruta" rel="noreferrer">',
 '</a>',
 '<em>',
 '<a href="http://pt.wikipedia.org/wiki/Keylogger" rel="noreferrer">',
 '</a>',
 '</em>',
 '<a href="http://pt.wikipedia.org/wiki/Criptoan%C3%A1lise_de_mangueira_de_borracha" rel="noreferrer">',
 '<em>',
 '</em>',
 '</a>',
 '<em>',
 '</em>',
 '</p>',
 '<p>',
 '</p>',
 '<p>',
 '<em>',
 '</em>',
 '</p>']

In [5]:
re.sub(r"<.*?>", "",df_pt["Questão"][0])

'Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?\n\nEstou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.\n\nTenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?\n\nQuais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?\n'

In [6]:
filter_tag = re.compile(r"<.*?>")
filter_tag.sub("",df_pt["Questão"][0])


'Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?\n\nEstou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.\n\nTenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?\n\nQuais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?\n'

## Function to remove caracter


In [7]:
def remove_caracter(text_value,regex_value):
    return regex_value.sub("",text_value)

In [8]:
def sub_code(text_value, regex_value):
    return regex_value.sub("code", text_value)


In [9]:
def sub_space(text_value, regex_value):
    return regex_value.sub(" ", text_value)

In [10]:
remove_caracter(df_pt["Questão"][0], filter_tag)


'Se eu fizer o hash de senhas antes de armazená-las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém?\n\nEstou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque, como força bruta na página de login da aplicação, keylogger no cliente e criptoanálise rubberhose. Qualquer forma de hash não vai impedir esses ataques.\n\nTenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido. Como dar maior garantia de segurança neste aspecto?\n\nQuais preocupações adicionais evitariam o acesso às senhas? Existem formas melhores de fazer esse hash?\n'

In [11]:
filter_code_tag = re.compile(r"<code>(.|(\n))*?</code>")
sub_code(df_en["Questão"][0], filter_code_tag)


'<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>\n\n<pre class="lang-cpp prettyprint-override">code</pre>\n\n<ul>\n<li>Without code, the code runs in 11.54 seconds.</li>\n<li>With the sorted data, the code runs in 1.93 seconds.</li>\n</ul>\n\n<p>Initially, I thought this might be just a language or compiler anomaly. So I tried it in Java.</p>\n\n<pre class="lang-java prettyprint-override">code</pre>\n\n<p>With a somewhat similar but less extreme result.</p>\n\n<hr>\n\n<p>My first thought was that sorting brings the data into the cache, but then I thought how silly that is because the array was just generated.</p>\n\n<ul>\n<li>What is going on?</li>\n<li>Why is it faster to process a sorted array than an unsorted array?</li>\n<li>The code is summing up some independent terms, and the order should not matter.</li>\n</ul>\n'

In [12]:
filter_punctuation_and_digit = re.compile(r"([^\w\s]|(\d+))")
filter_br = re.compile(r"(\n)")
filter_spaces = re.compile(r"\s+")


In [13]:
df_en["questions_cleaned"]=df_en.apply(lambda x: sub_space(remove_caracter(remove_caracter(sub_code(x["Questão"],filter_code_tag),filter_tag),filter_punctuation_and_digit),filter_br),axis=1).str.lower().replace(filter_spaces, ' ', regex=True)
df_pt["questions_cleaned"]=df_pt.apply(lambda x: sub_space(remove_caracter(remove_caracter(sub_code(x["Questão"],filter_code_tag),filter_tag),filter_punctuation_and_digit),filter_br),axis=1).str.lower().replace(filter_spaces, ' ', regex=True)
df_es["questions_cleaned"]=df_es.apply(lambda x: sub_space(remove_caracter(remove_caracter(sub_code(x["Questão"],filter_code_tag),filter_tag),filter_punctuation_and_digit),filter_br),axis=1).str.lower().replace(filter_spaces, ' ', regex=True)


In [14]:
display(df_en.head(5))
display(df_pt.head(5))
display(df_es.head(5))


,Id,Título,Questão,Tags,Pontuação,Visualizações,questions_cleaned
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,here is a piece of c code that seems very pecu...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,i accidentally committed the wrong files to gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,i want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,moderator note given that this question has a...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,ive been messing around with json for some tim...


,Id,Título,Questão,Tags,Pontuação,Visualizações,questions_cleaned
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,qual é a diferença entre code e code podem me ...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,é comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,me parecem termos muito próximos e eventualmen...


,Id,Título,Questão,Tags,Pontuação,Visualizações,questions_cleaned
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,las sentencias dinámicas son sentencias sql qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,cuál es la diferencia entre un code y un code ...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,la mayoría de nosotros decimos muchas veces si...
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,pregunta cómo validar un email que acepte todo...


In [15]:
text_test = "alura"
print(list(bigrams(text_test)))

[('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a')]


In [16]:
print(list(bigrams(pad_both_ends(text_test, n=2))))


[('<s>', 'a'), ('a', 'l'), ('l', 'u'), ('u', 'r'), ('r', 'a'), ('a', '</s>')]


In [17]:
df_en["language"] = "English"
df_pt["language"] = "Portuguese"
df_es["language"] = "Spanish"

In [18]:
rendom_state_value=123

question_en_train, question_en_test = train_test_split(
    df_en["questions_cleaned"], test_size=0.2, random_state=rendom_state_value)
question_pt_train, question_pt_test = train_test_split(
    df_pt["questions_cleaned"],test_size=0.2,random_state= rendom_state_value)
question_es_train, question_es_test = train_test_split(
    df_es["questions_cleaned"], test_size=0.2, random_state=rendom_state_value)

In [19]:
all_words_en = WhitespaceTokenizer().tokenize(' '.join(question_en_train))
all_words_pt = WhitespaceTokenizer().tokenize(' '.join(question_pt_train))
all_words_es = WhitespaceTokenizer().tokenize(' '.join(question_es_train))

print(f"[INFO] Size all_words_en: {len(all_words_en)}.")
print(f"[INFO] Size all_words_pt: {len(all_words_pt)}.")
print(f"[INFO] Size all_words_es: {len(all_words_es)}.")


[INFO] Size all_words_en: 21510.
[INFO] Size all_words_pt: 36716.
[INFO] Size all_words_es: 41248.


In [20]:
train_bigram_en, vocabulary_en = padded_everygram_pipeline(
    order=2, text=all_words_en)
train_bigram_pt, vocabulary_pt = padded_everygram_pipeline(
    order=2, text=all_words_pt)
train_bigram_es, vocabulary_es = padded_everygram_pipeline(
    order=2, text=all_words_es)


In [21]:
model_pt = MLE(2)
model_pt.fit(train_bigram_pt, vocabulary_pt)


In [22]:
model_pt.generate(num_words=6)

['e', 's', 'ã', 'o', '</s>', 'o']

In [23]:
model_pt.counts[['m']].items()

dict_items([('</s>', 2741), ('a', 2052), ('b', 144), ('p', 722), ('o', 854), ('e', 1503), ('u', 227), ('v', 16), ('í', 13), ('i', 290), ('ó', 37), ('â', 11), ('l', 23), ('é', 76), ('ú', 5), ('ã', 4), ('á', 32), ('s', 9), ('m', 25), ('g', 4), ('w', 1), ('_', 1), ('y', 10), ('d', 3)])

In [24]:
text_test = "bom dia"

words_bigramns = [
    list(bigrams(text_value))
    for text_value in [list(pad_both_ends(text_value, n=2)) for text_value in WhitespaceTokenizer().tokenize(text_test)]
]
words_bigramns


[[('<s>', 'b'), ('b', 'o'), ('o', 'm'), ('m', '</s>')],
 [('<s>', 'd'), ('d', 'i'), ('i', 'a'), ('a', '</s>')]]

In [25]:
# The closer to zero the better

print(model_pt.perplexity(words_bigramns[0]))
print(model_pt.perplexity(words_bigramns[1]))


15.299125474590992
8.118290440853844


In [26]:
def train_model_mle(list_texts):
    all_words = WhitespaceTokenizer().tokenize(' '.join(list_texts))
    all_words_bigrams, vocabulary = padded_everygram_pipeline(
        order=2, text=all_words)
    model = MLE(2)
    model.fit(all_words_bigrams, vocabulary)
    return model

In [27]:
model_en_mle = train_model_mle(question_en_train)
model_pt_mle = train_model_mle(question_pt_train)
model_es_mle = train_model_mle(question_es_train)


In [28]:
print(f"[INFO] model_en_mle perplexity: {model_en_mle.perplexity(words_bigramns[0])}")
print(f"[INFO] model_en_mle perplexity: {model_en_mle.perplexity(words_bigramns[1])}")
print(f"[INFO] model_pt_mle perplexity: {model_pt_mle.perplexity(words_bigramns[0])}")
print(f"[INFO] model_pt_mle perplexity: {model_pt_mle.perplexity(words_bigramns[1])}")
print(f"[INFO] model_es_mle perplexity: {model_es_mle.perplexity(words_bigramns[0])}")
print(f"[INFO] model_es_mle perplexity: {model_es_mle.perplexity(words_bigramns[1])}")


[INFO] model_en_mle perplexity: 13.207626654673446
[INFO] model_en_mle perplexity: 22.271365866014612
[INFO] model_pt_mle perplexity: 15.299125474590992
[INFO] model_pt_mle perplexity: 8.118290440853844
[INFO] model_es_mle perplexity: 43.952855947940236
[INFO] model_es_mle perplexity: 9.179377380281396


In [29]:
def calculate_perplexity(model, text):
    perplexity_sum = 0
    words_bigramns = [
        list(bigrams(text_value))
        for text_value in [list(pad_both_ends(text_value, n=2)) for text_value in WhitespaceTokenizer().tokenize(text)]
    ]
    for word in words_bigramns:
        perplexity_sum += model.perplexity(word)

    return perplexity_sum


In [30]:
calculate_perplexity(model_en_mle, text_test)


35.47899252068806

In [31]:
calculate_perplexity(model_pt_mle, question_pt_train.iloc[0])


379.69233961579755

In [32]:
calculate_perplexity(model_en_mle, question_pt_train.iloc[0])


inf

In [33]:
def train_model_laplace(list_texts):
    all_words = WhitespaceTokenizer().tokenize(' '.join(list_texts))
    all_words_bigrams, vocabulary = padded_everygram_pipeline(
        order=2, text=all_words)
    model = Laplace(2)
    model.fit(all_words_bigrams, vocabulary)
    return model


In [34]:
model_en_laplace = train_model_laplace(question_en_train)
model_pt_laplace = train_model_laplace(question_pt_train)
model_es_laplace = train_model_laplace(question_es_train)


In [35]:
print(
    f"[INFO] model_en_laplace perplexity: {calculate_perplexity(model_en_laplace, question_pt_train.iloc[0])}.")
print(
    f"[INFO] model_pt_laplace perplexity: {calculate_perplexity(model_pt_laplace, question_pt_train.iloc[0])}.")
print(
    f"[INFO] model_es_laplace perplexity: {calculate_perplexity(model_es_laplace, question_pt_train.iloc[0])}.")


[INFO] model_en_laplace perplexity: 1458.1098881610562.
[INFO] model_pt_laplace perplexity: 380.66088848279304.
[INFO] model_es_laplace perplexity: 693.2541308511308.


In [36]:
def assign_language(list_texts):
    language_list =[]
    for text_value in list_texts:
        english = calculate_perplexity(model_en_laplace,text_value) 
        portuguese = calculate_perplexity(model_pt_laplace,text_value) 
        spanish = calculate_perplexity(model_es_laplace,text_value) 
        if english>=portuguese <=spanish:
            language_list.append("Portuguese")
        elif portuguese>english<spanish:
            language_list.append("English")
        else:
            language_list.append("Spanish")
    return language_list



In [37]:
result_en = assign_language(question_en_test)
result_pt = assign_language(question_pt_test)
result_es = assign_language(question_es_test)


In [38]:
print(f"[INFO] Accuracy English: {100*result_en.count('English')/ len(question_en_test):.1f}%.")
print(f"[INFO] Accuracy Portuguese: {100*result_pt.count('Portuguese')/ len(question_pt_test):.1f}%.")
print(f"[INFO] Accuracy Spanish: {100*result_es.count('Spanish')/ len(question_es_test):.1f}%.")


[INFO] Accuracy English: 100.0%.
[INFO] Accuracy Portuguese: 100.0%.
[INFO] Accuracy Spanish: 97.0%.
